# Working with dbachecks Historical Results

This guide will assist you to get used to the database schema where dbachecks historical reports are stored if you have followed the blog post [dbachecks – Save the results to a database for historical reporting](https://sqldbawithabeard.com/2018/05/23/dbachecks-save-the-results-to-a-database-for-historical-reporting/)

The first thing that is required is to update the data in the database so that it reflects the current date as if dbachecks had been run today


## Demo Database and container

I have provided you with a demo backup which you can find in my [GitHub Presentations Repository](https://github.com/SQLDBAWithABeard/Presentations/tree/master/Notebooks/dbachecks-historical).

 I am also going to assume that you have Docker For Windows on your laptop (This will work in Docker running on other OS's but you will have to alter the paths accordingly). You will also find a docker-compose file in the [Repository](https://github.com/SQLDBAWithABeard/Presentations/blob/master/Notebooks/dbachecks-historical/docker-compose.yml) which you can use to spin up a SQL container and restore the backup. (Yes, I know I could have created a container with the database already embedded but this gives you more things to learn about!)

The docker-compose file looks like this
````
version: '3.7'

services:
    HistoricalReporting:
        image: mcr.microsoft.com/mssql/server:2019-CTP2.4-ubuntu
        ports:  
          - "15594:1433"
        environment:
          SA_PASSWORD: "Password0!"
          ACCEPT_EULA: "Y"
        volumes: 
          - C:\dbachecks:/var/opt/mssql/backups
````
It will create 

- a single service (a container (See the [Docker Documentation for a full definition of a service](https://docs.docker.com/get-started/part3/))) called `HistoricalReporting`
- assign port 1433 on the container to port 15594 on the host
- add the environmental variables for the SA Password and the acceptance of the EULA
- map the directory c:\dbachecks on the host (your laptop) to the directory /var/opt/mssql/backups in the container

It will create the container using the Community Technical Preview 2.4 image of SQL Server 2019 running on Ubuntu from the Microsoft Container Registry for SQL Server.

Download the backup file and the docker-compose.yml and save it in a folder, I am going to assume that you are using `C:\dbachecks` for the rest of this notebook.

You can, and should, understand how to change those values in the docker-compose file for different values so that you can spin up a container with a different image (maybe a later or earlier versions of SQL Server) or store your files in a different directory to `C:\dbachecks` or becuase you are allergic to `Password0!`

Once your docker-compose.yml file is ready open PowerShell (Windows or Core both will work) and type

Set-Location c:\dbachecks
docker-compose up -d

````
Set-Location c:\dbachecks
docker-compose up -d
````

<a href="command:workbench.action.terminal.focus">Or In Azure Data Studio you can click this link to open a new terminal'</a>

and then 

<a href="command:workbench.action.terminal.sendSequence?%7B%22text%22%3A%22Set-Location C:%5C%5Cdbachecks%22%7D">Paste in the Set-Location code by clicking this link</a> 

<a href="command:workbench.action.terminal.focus">Click this link to focus the terminal and press enter</a> 

<a href="command:workbench.action.terminal.sendSequence?%7B%22text%22%3A%22docker-compose up -d%22%7D">Paste in the docker-compose up -d code by clicking this link</a>

<a href="command:workbench.action.terminal.focus">Click this link to focus the terminal and press enter</a> 

You should see  
  
![alt text](https://github.com/SQLDBAWithABeard/Presentations/blob/master/Notebooks/dbachecks-historical/docker-compose-up.jpg?raw=true)




## Test that we have a running container

Lets check that you have got a running container that we can use

You will need to have dbatools installed - If you haven't Run

````
Install-Module dbatools
````
<a href="command:workbench.action.terminal.focus">In Azure Data Studio you can click this link to open a new terminal'</a>

and then 

<a href="command:workbench.action.terminal.sendSequence?%7B%22text%22%3A%22 %24cred%20%3D%20Get-Credential -Message%20%27Enter%20SA%20password %20for%20container%27%20sa%20%3B%20 if%28%28Test-DbaConnection%20-SqlInstance%20%27localhost%2C15594%27 -SqlCredential%20%24cred%29.ConnectSuccess%29%7BWrite-Output %27The%20Beard%20is%20Happy! - %20you%20may%20continue%27%7D else%7B%24err%20%3D%20%24error%5B0%5D.Exception.GetBaseException%28%29%20%3B%20Write-Warning%20%27The%20Beard%20is%20sad%2C%20we%20have%20an%20error%20-%20%24err%27%7D %22%7D">Paste in the Test-DbaConnection code</a> 

<a href="command:workbench.action.terminal.focus">Click this link to focus the terminal and press enter</a> 

In the credential prompt enter Password0! (or a different password if you have altered the docker-compose file)


## Restore the database from the backup

<a href="command:workbench.action.terminal.focus">Click this link to open a new terminal'</a>

and then 

<a href="command:workbench.action.terminal.sendSequence?%7B%22text%22%3A%22 %24cred%20%3D%20Get-Credential -Message%20%27Enter%20SA%20password %20for%20container%27%20sa %3BRestore-DbaDatabase -SqlInstance %27localhost%2C15594%27 -SqlCredential %24cred -Path %2Fvar%2Fopt%2Fmssql%2Fbackups%2FValidationResults-201942-13-13-57.bak -WithReplace %22%7D">Paste in the Restore-DBaDatabase code</a> 


<a href="command:workbench.action.terminal.focus">Focus the terminal and press enter</a> 

In the credential prompt enter Password0! (or a different password if you have altered the docker-compose file)

<img src="docker-compose-up.png">

In [1]:
/*Resets the date for the demo*/
USE [ValidationResults]
GO

UPDATE [dbachecks].[Prod_dbachecks_summary]
   SET [TestDate] = GETDATE()
 WHERE SummaryID = 7
GO

UPDATE [dbachecks].[Prod_dbachecks_summary]
   SET [TestDate] = DATEADD(day,-1,GetDAte())
 WHERE SummaryID = 6
GO

UPDATE [dbachecks].[Prod_dbachecks_summary]
   SET [TestDate] = DATEADD(day,-2,GetDAte())
 WHERE SummaryID = 5
 GO
 
UPDATE [dbachecks].[Prod_dbachecks_summary]
   SET [TestDate] = DATEADD(day,-3,GetDAte())
 WHERE SummaryID = 4
 GO
 
UPDATE [dbachecks].[Prod_dbachecks_summary]
   SET [TestDate] = DATEADD(day,-4,GetDAte())
 WHERE SummaryID = 3
 GO

 
UPDATE [dbachecks].[Prod_dbachecks_summary]
   SET [TestDate] = DATEADD(day,-5,GetDAte())
 WHERE SummaryID = 2
 GO

 
UPDATE [dbachecks].[Prod_dbachecks_summary]
   SET [TestDate] = DATEADD(day,-6,GetDAte())
 WHERE SummaryID = 1
 GO


Commands completed successfully.

Total execution time: 00:00:00.0309021

(1 row affected)

Total execution time: 00:00:00.0267746

(1 row affected)

...

Total execution time: 00:00:00.0123698

(1 row affected)

Total execution time: 00:00:00.0248558

(1 row affected)

Total execution time: 00:00:00.0131490

Lets list the Checks that we have and the descriptions

In [2]:
USE [ValidationResults]
GO
SELECT 
UniqueTag AS 'Check',  
Description
FROM dbachecks.Checks

Commands completed successfully.

Total execution time: 00:00:00.0066360

(89 rows affected)

Total execution time: 00:00:01.8335301

Check,Description
AgentServiceAccount,Tests that the SQL Agent Account is running and set to automatic
DbaOperator,Tests that the specified (default blank) DBA Operators exist and have the correct email address
FailsafeOperator,Tests that the specified (default blank) Failsafe Operator exists
DatabaseMailProfile,Tests that the specified (default blank) Database Mail Profile exists
FailedJob,Tests that enabled Agent Jobs last outcome was succeeded
ValidJobOwner,Testst that all Agent Jobs have a Job Owner in the list specified (default sa)
AgentAlert,Tests that there are Agent Alerts set up for the specified (default 16-25) alert severities and ids (default 823-825) and if specified Agent Jobs and/or notifications
DatabaseCollation,Tests that the Database Collation matches the instance collation except for specified databases
SuspectPage,Tests that there are 0 Suspect Pages for the database
TestLastBackup,Restores the last backup of a database onto a specified (default blank so will use the same instance) restore instance and checks if the DBCC result was succesful as well as the DBCC result. This can obviously take some time for large databases!


I want to see the total number of checks that passed and failed by date

In [3]:
USE [ValidationResults]
GO
SELECT 
TestDate,
TotalCount AS 'Total Checks',
PassedCount AS Passed,
FailedCount AS Failed,
SkippedCount AS Skipped
FROM dbachecks.Prod_dbachecks_summary


Commands completed successfully.

Total execution time: 00:00:00.0031211

(7 rows affected)

Total execution time: 00:00:00.0485921

TestDate,Total Checks,Passed,Failed,Skipped
2019-05-15,887,768,110,9
2019-05-16,904,795,100,9
2019-05-17,881,788,84,9
2019-05-18,892,787,96,9
2019-05-19,892,778,105,9
2019-05-20,880,843,22,15
2019-05-21,894,822,53,19


What are todays failures ?

In [4]:
SELECT 
d.Name,
ErrorRecord
FROM dbachecks.Prod_dbachecks_detail d
JOIN
dbachecks.Prod_dbachecks_summary s
ON
d.SummaryID = S.SummaryID
WHERE
s.TestDate = Convert(date, GetDate())
AND
d.Passed = 0
ORDER BY d.Name


(53 rows affected)

Total execution time: 00:00:00.1490683

Name,ErrorRecord
AdventureWorks2012 should return 0 database growth events on sql0,"Expected 0, because You want to control how your database files are grown, but got 1."
AdventureWorks2014 diff backups on sql0 Should Be less than 26 hours,"Expected 2018-05-25T07:29:44.7750989+00:00 to be greater than the actual value, because Taking regular backups is extraordinarily important, but got 2018-05-25T07:22:51.0000000."
AdventureWorks2014 file AdventureWorks2014_Data on SQL0 has free space under threshold,"Expected 20 to be greater or equal to the actual value, because free space within the file should be lower than threshold of 20 %, but got 6.486."
AdventureWorks2014 has a database compatibility level equal to the level of SQL0,"Expected 'Version140', because it means you are on the appropriate compatibility level for your SQL Server version to use all available features, but got Version120."
AdventureWorks2014 on sql0 should return 0 Disabled indexes,"Expected 0, because Disabled indexes are wasting disk space, but got 1."
AdventureWorks2016 diff backups on sql0 Should Be less than 26 hours,"Expected 2018-05-25T07:29:44.7991286+00:00 to be greater than the actual value, because Taking regular backups is extraordinarily important, but got 2018-05-25T07:22:52.0000000."
AdventureWorks2016 file AdventureWorks2016_Data on SQL0 has free space under threshold,"Expected 20 to be greater or equal to the actual value, because free space within the file should be lower than threshold of 20 %, but got 1.17400000000001."
AdventureWorks2016 has a database compatibility level equal to the level of SQL0,"Expected 'Version140', because it means you are on the appropriate compatibility level for your SQL Server version to use all available features, but got Version130."
AdventureWorks2017 diff backups on sql0 Should Be less than 26 hours,"Expected 2018-05-25T07:29:44.8130996+00:00 to be greater than the actual value, because Taking regular backups is extraordinarily important, but got 2018-05-25T07:22:52.0000000."
AdventureWorksDW2012 diff backups on sql0 Should Be less than 26 hours,"Expected 2018-05-25T07:29:44.8711028+00:00 to be greater than the actual value, because Taking regular backups is extraordinarily important, but got 2018-05-25T07:22:52.0000000."


Which instance had the most failures today ?

In [5]:
; With CTE AS (
SELECT 
DetailID,
RIGHT(Context, charindex(' ', reverse(Context) + ' ') - 1) AS SqlInstance
FROM dbachecks.Prod_dbachecks_detail
) 
SELECT 
CTE.SqlInstance,
COUNT(d.DetailID) AS 'Failed Checks'
FROM dbachecks.Prod_dbachecks_detail d
JOIN CTE
ON CTE.DetailID = d.DetailID
JOIN
dbachecks.Prod_dbachecks_summary s
ON
d.SummaryID = S.SummaryID
WHERE
s.TestDate = Convert(date, GetDate())
AND
d.Passed = 0
GROUP BY CTE.SqlInstance

(2 rows affected)

Total execution time: 00:00:00.1691171

SqlInstance,Failed Checks
sql0,39
sql1,14


Todays failures by instance and context

In [6]:
; With CTE AS (
SELECT 
DetailID,
RIGHT(Context, charindex(' ', reverse(Context) + ' ') - 1) AS SqlInstance
FROM dbachecks.Prod_dbachecks_detail
) 
SELECT 
CTE.SqlInstance,
d.Context,
COUNT(d.DetailID) AS 'Failed Checks'
FROM dbachecks.Prod_dbachecks_detail d
JOIN CTE
ON CTE.DetailID = d.DetailID
JOIN
dbachecks.Prod_dbachecks_summary s
ON
d.SummaryID = S.SummaryID
WHERE
s.TestDate = Convert(date, GetDate())
AND
d.Passed = 0
GROUP BY CTE.SqlInstance,d.Context
ORDER BY CTE.SqlInstance, [Failed Checks] DESC

(17 rows affected)

Total execution time: 00:00:00.0768573

SqlInstance,Context,Failed Checks
sql0,Testing last diff backups on sql0,9
sql0,Testing datafile growth type on sql0,8
sql0,Testing database compatibility level matches server compatibility level on sql0,6
sql0,Testing for files likely to grow soon on sql0,5
sql0,Testing Linked Servers on sql0,4
sql0,Testing database growth event on sql0,2
sql0,Checking error log on sql0,1
sql0,Database status is correct on sql0,1
sql0,Testing Network Latency on sql0,1
sql0,Testing Disabled indexes on sql0,1


Number of failures by date, instance and context

In [7]:
; With CTE AS (
SELECT 
DetailID,
RIGHT(Context, charindex(' ', reverse(Context) + ' ') - 1) AS SqlInstance
FROM dbachecks.Prod_dbachecks_detail
) 
SELECT 
s.TestDate,
CTE.SqlInstance,
d.Context,
COUNT(d.DetailID) AS 'Failed Checks'
FROM dbachecks.Prod_dbachecks_detail d
JOIN CTE
ON CTE.DetailID = d.DetailID
JOIN
dbachecks.Prod_dbachecks_summary s
ON
d.SummaryID = S.SummaryID
WHERE
d.Passed = 0
GROUP BY s.TestDate,CTE.SqlInstance,d.Context
ORDER BY CTE.SqlInstance, [Failed Checks] DESC

(153 rows affected)

Total execution time: 00:00:00.0971087

TestDate,SqlInstance,Context,Failed Checks
2019-05-15,sql0,Testing datafile growth type on sql0,14
2019-05-16,sql0,Testing datafile growth type on sql0,14
2019-05-17,sql0,Testing datafile growth type on sql0,14
2019-05-18,sql0,Testing datafile growth type on sql0,14
2019-05-19,sql0,Testing datafile growth type on sql0,14
2019-05-15,sql0,Testing last full backups on sql0,13
2019-05-16,sql0,Testing last diff backups on sql0,10
2019-05-15,sql0,Testing last diff backups on sql0,10
2019-05-19,sql0,Testing last diff backups on sql0,10
2019-05-21,sql0,Testing last diff backups on sql0,9


This is just a quick example of some of the things in the dbachecks ValidationResults database - What more can you do ?

<a href="command:workbench.action.terminal.focus">Or In Azure Data Studio you can click this link to open a new terminal'</a>

and then 

<a href="command:workbench.action.terminal.sendSequence?%7B%22text%22%3A%22 %24cred %3D Get-Credential %3BRestore-DbaDatabase -SqlInstance %27localhost%2C15594%27 -SqlCredential %24cred -Path %2Fvar%2Fopt%2Fmssql%2Fbackups%2FValidationResults-201942-13-13-57.bak %22%7D">Paste in the Restore-DBaDatabase code by clicking this link - :-)</a> 


<a href="command:workbench.action.terminal.sendSequence?%7B%22text%22%3A%22docker-compose up -d%22%7D">Paste in the docker-compose up -d code by clicking this link</a>

<a href="command:workbench.action.terminal.focus">Click this link to focus the terminal and press enter</a> 

In the credential prompt enter Password0! (or a different password if you have altered the docker-compose file)